# Demo: `transform`


*Python* bilježnica pred tobom služi za demonstraciju korištenja funkcije `script.feature_engineering.transform`. Autor bilježnice isti je kao i autor funkcije (i cijele skripte `script.feature_engineering`) &mdash; Davor Penzar.


In [1]:
import matplotlib as mlp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from script.feature_engineering import *


Proučimo prvo dokumentaciju funkcije.


In [2]:
help(transform)


Help on function transform in module script.feature_engineering:

transform(df, transformers)
    Konstruiraj tablicu dobivenu transformacijom originalne tablice.
    
    Povratna tablica dobivena je konkatenacijom po stupcima povratnih
    vrijednosti poziva
        >>> transformer(df[column])
    za svaki uredeni par (column, transformer) u objektu transformers (dict ili
    lista uredenih parova).  Specijalno, transformer moze biti i None cime se
    stupac samo "prepisuje" u povratnu tablicu bez transformacije, to jest,
    to je ekvivalentno sa slucajem da je transformer "identiteta".
    
    Moguce je za kljuc transformacije --- columns u uredenom paru
    (columns, transformer) u objektu transformers --- zadati i listu stupaca.
    Na primjer, legalno je
        >>> df = pandas.DataFrame({'col1' : [0, 1, 0], 'col2' : [1, 1, 1]})
        >>> transformers(df, [('col1', None), (['col1', 'col2'], lambda cols : cols['col1'] - cols['col2'])])
           col1  0
        0     0 -1
  

Za brzu demonstraciju funkcije, izmislit ćemo tablicu podataka. Neka ona bude ocjena općeg dojma mjeseca u nekoj godini na nekome mjestu. Ocjenjivači će također biti anonimni, zadani šifrirano kao *Sudac A*, *Sudac B*, *Sudac C*. Osim ocjena, predstavit ćemo i temperaturu u stupnjevima Celzijevim; neka ona, na primjer, predstavlja srednju temperaturu u mjesecu. Zajedno nastupanje ocjena i temperatura u tablici ne znači da se ocjene odnose na temperaturu ili vremenske prilike uopće &mdash; možda se ocjene odnose na nešto sasvim drugačije, a znanstvenike zanima eventualna korelacija temperature i ispitanog dojma.

Mjeseci će biti enumerirani brojevima od $ 1 $ do $ 12 $, a temperatura će u tom modelu otprilike pratiti formulu
$$ T \left( t \right) = 14 + 15 \sin \left( \frac{\pi}{6} \left( t - 4 \right) \right) \text{.} $$
Odstupanje od egzaktnog rezultata formule bit će normalno distribuirano s očekivanjem $ 0 $ i varijancom $ 2.5^{2} $. Dodatno, temperatura je zaokružena na jedno decimalno mjesto.

Ocjene će biti cjelobrojne s najnižom ocjenom $ 1 $, a najvišom $ 10 $. Generirat ćemo ih slučajno, tako da budu normalno distribuirane s očekivanjem $ 5.5 $ i varijancom $ 2.5^{2} $, međutim, ta *normalna distribucija* ocjena ne će u stvarnosti biti normalna jer su ocjene nužno u rasponu od $ 1 $ do $ 10 $ i cjelobrojne.


In [3]:
# Generiranje temperature.
temp = (
    14 + 15 * np.sin(np.pi / 6 * (np.arange(1, 13, dtype = float) - 4)) +
    2.5 * np.random.randn(12)
).round(1)

# Generiranje ocjena.
mark = np.minimum(
    np.maximum(
        np.round(5.5 + 2.5 * np.random.randn(3, 12)).astype(int),
        1
    ),
    10
)

# Generiranje tablice.
data = pd.DataFrame(
    data = np.concatenate((temp.reshape(1, 12), mark), axis = 0).T,
    index = np.arange(1, 13),
    columns = ['Temperatura', 'Sudac A', 'Sudac B', 'Sudac C']
)
data[['Sudac A', 'Sudac B', 'Sudac C']] = data[['Sudac A', 'Sudac B', 'Sudac C']].astype(int)

# Oslobađanje memorije.
del temp
del mark


Prikažimo dobivenu tablicu pdataka.


In [4]:
data


,Temperatura,Sudac A,Sudac B,Sudac C
1,-4.2,4,6,1
2,-4.3,2,1,6
3,10.3,3,5,8
4,10.4,5,1,5
5,17.1,7,3,4
6,27.7,6,7,6
7,30.1,5,3,2
8,28.4,5,8,6
9,19.7,4,4,8
10,11.8,3,2,4


Od ove tablice načinit ćemo novu tablicu:

1.  Temperatura će biti izražena u Fahrenheitima.
2.  Osim originalnih os+cjena sudaca, izrazit ćemo najnižu, najvišu i prosječnu ocjenu sudaca.

U tu svrhu, definirajmo funkciju trasformacije temperature iz stupnjeva Celzijevih u Fahrenheite. Potpunosti radi, definirat ćemo i njezin inverz, a, kako je ona afina injekcija, inverz nije ništa kompleksniji od početne (ionako jednostavne) formule.


In [5]:
def cel2far (t):
    """Temperaturu u stupnjevima C izrazi u Fahrenheitima."""
    return 9.0 / 5.0 * t + 32

def far2cel (t):
    """Temperaturu u Fahrenheitima izrazi u stupnjevima C."""
    return 5.0 / 9.0 * (t - 32)


Novu ćemo tablicu generirati pozivom funkcije `transform`. Prvi argument, argument `df`, bit će naša tablica `data`, a za drugi argument proslijedit ćemo listu čiji su svi elementi objekti klase `tuple`. Na prvom mjestu (na indeksu $ 0 $) tih `tuple`-ova bit će naziv stupca kao *string* (ili lista &mdash; `list` &mdash; naziva stupaca) čiju transformaciju tražimo, a na drugom mjestu (na indeksu $ 1 $) funkcija kojom stupce transformiramo. Kao što je u dokumentaciji predstavljeno, doslovni prijepis stupaca *Sudac A*, *Sudac B* i *Sudac C* izvršit ćemo prosljeđivanjem objekta `None` umjesto funkcijskog objekta u drugom `tuple`-u (na indeksu $ 1 $) proslijeđene liste.


In [6]:
new_data = transform(
    data,
    [
        ('Temperatura', lambda t : cel2far(t).round(1)),
        (['Sudac A', 'Sudac B', 'Sudac C'], None),
        (['Sudac A', 'Sudac B', 'Sudac C'], lambda df : df.min(axis = 1)),
        (['Sudac A', 'Sudac B', 'Sudac C'], lambda df : df.max(axis = 1)),
        (['Sudac A', 'Sudac B', 'Sudac C'], lambda df : df.mean(axis = 1).round(2))
    ]
)
new_data.columns = new_data.columns[:-3].tolist() + ['Min', 'Max', 'Sredina']


**Nota bene.** U listi uređenih parova, pa prvom mjestu (na indeksu $ 0 $) para vrijednost je `x` takva da se od argumenta `df` funkciji transformacije (ako ona nije `None`) prosljeđuje `df[x]`. U danom primjeru od tri stupca nije bilo potrebno imenovati ih za traženje ekstrema i srednje vrijednosti, ali, na primjer, ako bismo od nove tablice radili i treću u kojoj bismo računali raspon ocjena, to bismo mogli učiniti pozivom
```Python
transform(
    new_data,
    [
        # ...
        (['Min', 'Max'], lambda df : df['Max'] - df['Min']),
        # ...
    ]
)
```
Dakle, ako funkciji transformacije prosljeđujemo više stupaca, oni njoj nisu proslijeđeni kao toliko argumenata koliko je stupaca, nego doslovno kao `df[x]` &mdash; rezultantni argument je ponovo objekt klase `pandas.DataFrame` čiji su stupci one koje smo proslijedili (i u tom poretku u kojem smo ih proslijedili).


Za funkcijski objekt za transformaciju stupca *Temperatura* nije bilo potrebno definirati *lambda*-funkciju, mogli smo proslijediti doslovno funkciju `cel2far`, ali, konsistencije radi, zahtijevat ćemo da su temperature i u tom stupcu zaokružene na jedno decimalno mjesto. Upravo zbog tog zaokruživanja koristimo *lambda*-funkciju.

Proučimo novu tablicu.


In [7]:
new_data


,Temperatura,Sudac A,Sudac B,Sudac C,Min,Max,Sredina
1,24.4,4,6,1,1,6,3.67
2,24.3,2,1,6,1,6,3.00
3,50.5,3,5,8,3,8,5.33
4,50.7,5,1,5,1,5,3.67
5,62.8,7,3,4,3,7,4.67
6,81.9,6,7,6,6,7,6.33
7,86.2,5,3,2,2,5,3.33
8,83.1,5,8,6,5,8,6.33
9,67.5,4,4,8,4,8,5.33
10,53.2,3,2,4,2,4,3.00


## Napomene za optimizaciju


Nekada za više različitih transformacija koristimo iste međurezultate; nekada nas osim završnih rezultata zanimaju i ti međurezultati. U velikim tablicama stoga nije efikasno za svaku transformaciju (među)rezultate računati ispočetka, nego ih je efikasnije izračunati samo jednom. Na primjer, ako želimo prikazati korijen i logaritam korijena vrijednosti u nekom stupcu neke velike tablice (ni sam ne znam pravu situaciju kada bismo to točno radili), i ako bismo zanemarili naše znanje iz matematike zbog kojeg smo svjesni da za logaritam korijena nije potrebno računati korijen pa njegov logaritam, nego samo logaritam i podijeliti ga s $ 2 $, za oba stupca trebali bismo računati korijen što vjerojatno nije toliko jeftina operacija.

Optimizacija spomenutog problema na način da korijen računamo samo jednom moguća je. U spoemutom primjeru, za transformaciju stupca u pozivu funkcije `transform` prosljeđujemo funkciju
```Python
def sqrt_logsqrt (x):
    """Dohvati korijen i logaritam korijena od x."""

    y = np.sqrt(x)
    z = np.log(y)

    # (preienovanje stupaca y, z)...

    return pd.concat((y, z), axis = 1)
```
Time ćemo u konačnoj tablici uistinu dobiti tražena dva stupca, a pri njihovom računanju korijen ćemo računati samo jednom. Efikasno, zar ne?
